In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import requests

# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 64
n_layer = 256
dropout = 0.1
# ------------

torch.manual_seed(1337)

text = requests.get(url='https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt')

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y[0]

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


class TransformerModel(nn.Module):
    def __init__(self, vocab_size, n_embd, n_head, n_layer, block_size, dropout=0.1):
        super().__init__()
        self.vocab_size = vocab_size
        self.block_size = block_size
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)
        # Token and position embeddings
        self.token_embeddings = nn.Embedding(vocab_size, n_embd)
        self.position_embeddings = nn.Parameter(torch.zeros(1, block_size, n_embd))

        # Transformer blocks
        self.transformer_blocks = nn.ModuleList([
            nn.TransformerEncoderLayer(
                d_model=n_embd,
                nhead=n_head,
                dim_feedforward=4 * n_embd,
                dropout=dropout,
                activation='gelu'
            ) for _ in range(n_layer)
        ])

        # Layer norm before the final linear layer
        self.ln_f = nn.LayerNorm(n_embd)

        # Decoder head
        self.head = nn.Linear(n_embd, vocab_size, bias=False)

    def forward(self, idx, targets=None):
        B, T = idx.size()
        assert T <= self.block_size, "Cannot forward, model block size is exhausted."

        # Token and position embeddings
        token_embeddings = self.token_embeddings(idx)
        position_embeddings = self.position_embeddings[:, :T, :]
        x = token_embeddings + position_embeddings

        # Apply Transformer blocks
        for block in self.transformer_blocks:
            x = block(x)

        x = self.ln_f(x)
        logits = self.head(x)

        # Calculate loss if targets are provided
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))

        return logits, loss
    def generate(self, input_idx, max_length=50):
        self.eval()  # Set the model to evaluation mode
        with torch.no_grad():  # No need to track gradients
            for _ in range(max_length):
                logits, _ = self(input_idx)
                # Get the logits of the last token in the sequence
                probabilities = F.softmax(logits[:, -1, :], dim=-1)
                # Sample from the distribution or pick the most likely
                next_token = torch.multinomial(probabilities, num_samples=1)
                # Append to the sequence and use as the next input
                input_idx = torch.cat([input_idx, next_token], dim=-1)

                # Check if the last token is a stopping criterion, e.g., end of sequence token
                if next_token.item() == stoi['<EOS>']:  # Assuming <EOS> is your end-of-sequence token
                    break

        return input_idx
# Model parameters
vocab_size = len(chars)  # Assuming `chars` is defined as before
n_embd = 256
n_head = 8
n_layer = 6
block_size = 128
dropout = 0.1
batch_size = 16348

# Initialize model
model = TransformerModel(vocab_size, n_embd, n_head, n_layer, block_size, dropout)
model.train()

# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)



m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

#PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))
model.summary()

18.471435 M parameters


In [ ]:

# Training loop
# Training loop
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    knowledge_base_dir = ('C:/Users/jonny/Documents/PATH/ONI/knowledge_base/remembered_texts/')
    train_data = os.path.join(knowledge_base_dir, 'train.txt')
    val_data = os.path.join(knowledge_base_dir, 'val.txt')
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y[0]

for epoch in range(10):  # Assuming a suitable number of epochs
    for xb, yb in get_batch('train'):  # Assuming a data loader is defined
        optimizer.zero_grad()
        logits, loss = model(xb, yb)
        loss.backward()
        optimizer.step()
        print(f"Loss: {loss.item()}")

In [ ]:
import numpy as np
import tensorflow as tf
import tkinter as tk
from tkinter import filedialog

import cv2
import os

# Placeholder functions for your custom model's text processing
def process_input_text(input_text):
    # Implement this to convert input_text to the format your model expects
    # For example, if the model expects tokenized input, tokenize the text here
    return input_text

def generate_response(model, text_input, image_input=None, audio_input=None, video_input=None):
    # Ensure that we provide four inputs to the model
    if image_input is None:
        image_input = np.zeros((1, 1920, 1080, 3), dtype=np.float32)  # Dummy image input
    if audio_input is None:
        audio_input = np.zeros((1, 1000, 1), dtype=np.float32)        # Dummy audio input
    if video_input is None:
        video_input = np.zeros((1, 225, 225, 3), dtype=np.float32)    # Dummy video input

    text_input = np.array([text_input])  # Ensure text input is in array form
    inputs = [text_input, image_input, audio_input, video_input]
    response = model.predict(inputs)
    return response


# Function to handle video capture
def capture_video_frame():
    cap = cv2.VideoCapture(0)
    ret, frame = cap.read()
    cap.release()
    if ret:
        frame = cv2.resize(frame, (225, 225))  # Resize the frame to match the expected size
        return frame
    else:
        return np.zeros((225, 225, 3), dtype=np.float32)  # Dummy video input


os.environ['DISPLAY'] = ':1'

# Initialize the Tkinter root
root = tk.Tk()
root.title("OniChat")

# Create the conversation display
conversation_frame = tk.Frame(root)
conversation_frame.pack(pady=10)

conversation_list = tk.Listbox(conversation_frame, width=80, height=20)
conversation_list.pack(side=tk.LEFT, fill=tk.BOTH)

scrollbar = tk.Scrollbar(conversation_frame, orient=tk.VERTICAL)
scrollbar.config(command=conversation_list.yview)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

conversation_list.config(yscrollcommand=scrollbar.set)

# Function to handle user input and generate response
def send_message(event=None):
    user_input = user_entry.get()
    if user_input:
        conversation_list.insert(tk.END, f"You: {user_input}")
        user_entry.delete(0, tk.END)
        
        # Process the input and generate response using the custom model
        processed_input = process_input_text(user_input)
        
        # Capture video frame
        video_frame = capture_video_frame()
        
        response_text = generate_response(trans_model, processed_input, video_input=video_frame)
        
        conversation_list.insert(tk.END, f"Bot: {response_text}")
        conversation_list.yview(tk.END)  # Scroll to the bottom

# Create the entry widget for user input
user_entry = tk.Entry(root, width=80)
user_entry.pack(pady=10)
user_entry.bind("<Return>", send_message)

# Create the send button
send_button = tk.Button(root, text="Send", command=send_message)
send_button.pack()

def read_pdf(file_path):
    try:
        with open(file_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            for page_num in range(len(reader.pages)):
                page = reader.pages[page_num]
                print(page.extract_text())
    except Exception as e:
        print(f"Error reading PDF file {file_path}: {e}")

def read_docx(file_path):
    try:
        doc = Document(file_path)
        for para in doc.paragraphs:
            print(para.text)
    except Exception as e:
        print(f"Error reading DOCX file {file_path}: {e}")

# Function to handle file upload
def upload_file():
    file_path = filedialog.askopenfilename()
    if file_path:
        conversation_list.insert(tk.END, f"File uploaded: {file_path}")
        if file_path.endswith('.pdf'):
            read_pdf(file_path)
        elif file_path.endswith('.docx'):
            read_docx(file_path)
        else:
            with open(file_path, 'r', encoding='utf-8') as file:
                data = file.read()
                conversation_list.insert(tk.END, f"File content: {data[:100]}...")  # Display first 100 characters
                processed_input = process_input_text(data)
                response_text = generate_response(trans_model, processed_input)
                conversation_list.insert(tk.END, f"Bot: {response_text}")

# Create the file upload button
upload_button = tk.Button(root, text="Upload File", command=upload_file)
upload_button.pack()

# Initialize the Ears class with a proper callback
ears = Ears(audio_callback=Ears.audio_callback)

# Example usage
read_pdf('C:/Users/jonny/Documents/PATH/ONI/knowledge_base/remembered_texts/')
read_docx('C:/Users/jonny/Documents/PATH/ONI/knowledge_base/remembered_texts/')
# Function to handle microphone input
def record_audio(ears):
    conversation_list.insert(tk.END, "Listening...")
    ears = Ears
    ears.start_listening()
    time.sleep(10)  # Listen for 10 seconds (adjust as needed)
    ears.stream.stop()
    conversation_list.insert(tk.END, "Stopped listening")


# Create the microphone button
microphone_button = tk.Button(root, text="Microphone", command=record_audio)
microphone_button.pack()

# Training loop to use files from ./knowledge_base
def training_loop(model, knowledge_base_dir):
    for filename in os.listdir(knowledge_base_dir):
        file_path = os.path.join(knowledge_base_dir, filename)
        if os.path.isfile(file_path):
            try:
               with open(file_path, 'r', encoding='utf-8') as file:
                    data = file.read()
                    processed_input = process_input_text(data)
                    # We will use dummy values for other inputs during training
            except UnicodeDecodeError:
                    print(f"Error decoding file: {file_path}")
                    processed_input = process_input_text(file_path)
                    # We will use dummy values for other inputs during training
                    image_input = np.zeros((1, 1920, 1080, 3), dtype=np.float32)
                    audio_input = np.zeros((1, 1000, 1), dtype=np.float32)
                    video_input = np.zeros((1, 225, 225, 3), dtype=np.float32)
                                
                # Train the model
                #model.train_on_batch([processed_input, image_input, audio_input, video_input], processed_input)  # Adjust this line based on your model's training method

# Path to the knowledge base directory

knowledge_base_dir = ('C:/Users/jonny/Documents/PATH/ONI/knowledge_base/remembered_texts/')
# Start the training loop
training_loop(model, knowledge_base_dir)

# Run the Tkinter main loop
root.mainloop()